<a href="https://colab.research.google.com/github/slala2121/Triplet-net-keras/blob/COS597D/deep_metric_learning_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code adapted from:
https://github.com/KinWaiCheuk/Triplet-net-keras

Other relevant links:

scratch classification network from https://keras.io/examples/cifar10_resnet/

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
!pip install --upgrade tensorflow
!pip install tensorflow-addons


Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [7]:
# current work around for fixing the lifted structure loss file

%%writefile /usr/local/lib/python3.6/dist-packages/tensorflow_addons/losses/lifted.py


# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Implements lifted_struct_loss."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow_addons.losses import metric_learning
from tensorflow_addons.utils import keras_utils


@keras_utils.register_keras_custom_object
@tf.function
def lifted_struct_loss(labels, embeddings, margin=1.0):
    """Computes the lifted structured loss.

    Args:
      labels: 1-D tf.int32 `Tensor` with shape [batch_size] of
        multiclass integer labels.
      embeddings: 2-D float `Tensor` of embedding vectors. Embeddings should
        not be l2 normalized.
      margin: Float, margin term in the loss definition.

    Returns:
      lifted_loss: tf.float32 scalar.
    """
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    lshape = tf.shape(labels)
    # assert lshape.shape == 1
    labels = tf.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pairwise_distances = metric_learning.pairwise_distance(embeddings)

    # Build pairwise binary adjacency matrix.
    adjacency = tf.math.equal(labels, tf.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = tf.math.logical_not(adjacency)

    batch_size = tf.size(labels)

    diff = margin - pairwise_distances
    mask = tf.cast(adjacency_not, dtype=tf.dtypes.float32)
    # Safe maximum: Temporarily shift negative distances
    #   above zero before taking max.
    #     this is to take the max only among negatives.
    row_minimums = tf.math.reduce_min(diff, 1, keepdims=True)
    row_negative_maximums = tf.math.reduce_max(
        tf.math.multiply(diff - row_minimums, mask), 1,
        keepdims=True) + row_minimums

    # Compute the loss.
    # Keep track of matrix of maximums where M_ij = max(m_i, m_j)
    #   where m_i is the max of alpha - negative D_i's.
    # This matches the Caffe loss layer implementation at:
    #   https://github.com/rksltnl/Caffe-Deep-Metric-Learning-CVPR16/blob/0efd7544a9846f58df923c8b992198ba5c355454/src/caffe/layers/lifted_struct_similarity_softmax_layer.cpp  # pylint: disable=line-too-long

    max_elements = tf.math.maximum(row_negative_maximums,
                                   tf.transpose(row_negative_maximums))
    diff_tiled = tf.tile(diff, [batch_size, 1])
    mask_tiled = tf.tile(mask, [batch_size, 1])
    max_elements_vect = tf.reshape(tf.transpose(max_elements), [-1, 1])

    loss_exp_left = tf.reshape(
        tf.math.reduce_sum(
            tf.math.multiply(
                tf.math.exp(diff_tiled - max_elements_vect), mask_tiled),
            1,
            keepdims=True), [batch_size, batch_size])

    loss_mat = max_elements + tf.math.log(loss_exp_left +
                                          tf.transpose(loss_exp_left))
    # Add the positive distance.
    loss_mat += pairwise_distances

    mask_positives = tf.cast(
        adjacency, dtype=tf.dtypes.float32) - tf.linalg.diag(
            tf.ones([batch_size]))

    # *0.5 for upper triangular, and another *0.5 for 1/2 factor for loss^2.
    num_positives = tf.math.reduce_sum(mask_positives) / 2.0

    lifted_loss = tf.math.truediv(
        0.25 * tf.math.reduce_sum(
            tf.math.square(
                tf.math.maximum(
                    tf.math.multiply(loss_mat, mask_positives), 0.0))),
        num_positives)
    return lifted_loss


@keras_utils.register_keras_custom_object
class LiftedStructLoss(tf.keras.losses.Loss):
    """Computes the lifted structured loss.

    The loss encourages the positive distances (between a pair of embeddings
    with the same labels) to be smaller than any negative distances (between
    a pair of embeddings with different labels) in the mini-batch in a way
    that is differentiable with respect to the embedding vectors.
    See: https://arxiv.org/abs/1511.06452.

    Args:
      margin: Float, margin term in the loss definition.
      name: Optional name for the op.
    """

    def __init__(self, margin=1.0, name=None):
        super(LiftedStructLoss, self).__init__(
            name=name, reduction=tf.keras.losses.Reduction.NONE)
        self.margin = margin

    def call(self, y_true, y_pred):
        return lifted_struct_loss(y_true, y_pred, self.margin)

    def get_config(self):
        config = {
            "margin": self.margin,
        }
        base_config = super(LiftedStructLoss, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



Overwriting /usr/local/lib/python3.6/dist-packages/tensorflow_addons/losses/lifted.py


In [0]:
import os
os.kill(os.getpid(), 9)

In [0]:
import time
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import random
from itertools import permutations
from PIL import Image
import cv2

import tensorflow as tf

from tensorflow import keras

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optimizers

from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import binary_crossentropy

import tensorflow_addons as tfa


source_path=os.path.join('drive','My Drive', 'Colab Notebooks')


In [0]:
# prepare dataset either for classification or deep metric learning
import tensorflow_datasets as tfds

dataset_name='cifar10'

dataset_dir=os.path.join(source_path,dataset_name)
if not os.path.isdir(dataset_dir):
  os.mkdir(dataset_dir)

debug=1

if debug:
  split_percent=1
  train_split = tfds.Split.TRAIN.subsplit(tfds.percent[:split_percent])
  test_split = tfds.Split.TEST.subsplit(tfds.percent[:split_percent])
  train_dataset,info = tfds.load(name=dataset_name, split=train_split, as_supervised=True, with_info=True)
  test_dataset,info = tfds.load(name=dataset_name, split=test_split, as_supervised=True, with_info=True)
else:
  train_dataset,info = tfds.load(name=dataset_name, split='train', as_supervised=True, with_info=True)
  test_dataset,info = tfds.load(name=dataset_name, split='test', as_supervised=True, with_info=True)

input_dim=info.features['image'].shape
num_classes=info.features['label'].num_classes

train_mean_path=os.path.join(dataset_dir,'train_mean.npy')
if os.path.exists(train_mean_path):
  train_mean=np.load(train_mean_path)
else:
  train_mean=[]
  num_train_images=info.splits['train'].num_examples
  train_mean=[]
  for example in train_dataset.take(num_train_images):
    image,label=example[0],example[1]
    image=image.numpy().astype('float32')
    if len(train_mean)==0:
      train_mean=image
    else:
      train_mean = train_mean+image

  train_mean=train_mean*1.0/num_train_images
  np.save(train_mean_path,train_mean)








In [0]:
# prepare dataset either for classification or deep metric learning

def _normalize_img(img, label):
    img = img - train_mean
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)


# preprocessing of labels for classification

def _encode_one_hot(img, label):
    label = tf.one_hot(label,num_classes)
    return (img, label)


train_dataset = train_dataset.map(_normalize_img)
test_dataset = test_dataset.map(_normalize_img)

loss_type='lifted'
if loss_type=='classification':
  train_dataset = train_dataset.map(_encode_one_hot)
  test_dataset = test_dataset.map(_encode_one_hot)

# Build your input pipelines

train_dataset = train_dataset.shuffle(1024).batch(32)
test_dataset = test_dataset.batch(32)

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

# based on the lifted scheme paper
def create_deep_base_network(input_dim,loss_type,num_classes=0,transfer=False,freeze_weights=False):
  weights='imagenet' if transfer else None
  conv_base = ResNet50(weights=weights, include_top=False, input_shape=input_dim)

  if freeze_weights:
    for layer in conv_base.layers:
      layer.trainable=False
  
  model = models.Sequential()
  model.add(layers.Input(input_dim))
  model.add(conv_base)

  model.add(layers.Flatten())
  model.add(layers.BatchNormalization())

  if loss_type=='classification':
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation='softmax'))
  else:
    model.add(layers.Dense(64, activation=None))
    model.add(layers.Dropout(0.5))

  return model


def create_shallow_network(input_dim,loss_type,num_classes=0):
  model=tf.keras.Sequential()
  model.add(layers.Input(input_dim))
  model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same'))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.BatchNormalization())

  if loss_type=='classification':
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers(Dense(num_classes,activation='softmax')))
  else:
    model.add(tf.keras.layers.Dense(256, activation=None))
    model.add(tf.keras.layers.Dropout(0.5))

  return model

def construct_model(model_type,input_dim,loss_type,num_classes,transfer=False,freeze_weights=False):
  if model_type=='shallow':
    model=create_shallow_network(input_dim,loss_type,num_classes)
  elif model_type=='deep':
    model=create_deep_base_network(input_dim,loss_type,num_classes,transfer,freeze_weights)

  if loss_type =='triplet':
    model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))
  return model

def construct_loss(loss_type,margin):
  if loss_type=='triplet':
    loss=tfa.losses.TripletSemiHardLoss(margin=margin)
  elif loss_type=='lifted':
    loss=tfa.losses.LiftedStructLoss(margin=margin)
  elif loss_type=='classification':
    loss=tf.keras.losses.CategoricalCrossentropy()
  return loss

In [5]:
# tune lr

lrs=[1e-1,1e-2,1e-3,1e-4,1e-5]
lrs=[1e-4]
model_type='deep'
transfer=True
freeze_weights=False

margin=1.0

model_dir=os.path.join(dataset_dir,model_type)
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)

num_epochs=30

fig,ax=plt.subplots(2,3)
ax=ax.ravel()
for lr_index,lr in enumerate(lrs):

  model=construct_model(model_type,input_dim,loss_type,num_classes,transfer,freeze_weights)
  loss=construct_loss(loss_type,margin)

  # Compile the model
  model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss=loss)

  history = model.fit(
      train_dataset,
      epochs=num_epochs)

  ax[lr_index].set_title('Loss for lr_%s'%(str(lr)))
  ax[lr_index].plot(np.arange(num_epochs),history.history['loss'],'r',label='train_loss_lr_%s'%(str(lr)))
  
plt.legend()
plt.savefig(os.path.join(model_dir,'loss_plot_lr_tune_%s_transfer_%s_freeze_%s_%s_%s.png'%(model_type,transfer,freeze_weights,loss_type,margin)))
plt.close()

Epoch 1/30
16/16 [==============================] - 11s 706ms/step - loss: 39.6720
Epoch 2/30
16/16 [==============================] - 2s 115ms/step - loss: 36.2597
Epoch 3/30
16/16 [==============================] - 2s 110ms/step - loss: 32.1725
Epoch 4/30
16/16 [==============================] - 2s 112ms/step - loss: 31.2699
Epoch 5/30
16/16 [==============================] - 2s 114ms/step - loss: 29.6449
Epoch 6/30
16/16 [==============================] - 2s 113ms/step - loss: 28.6109
Epoch 7/30
16/16 [==============================] - 2s 111ms/step - loss: 27.3904
Epoch 8/30
16/16 [==============================] - 2s 119ms/step - loss: 27.8023
Epoch 9/30
16/16 [==============================] - 2s 113ms/step - loss: 28.2717
Epoch 10/30
16/16 [==============================] - 2s 116ms/step - loss: 26.7183
Epoch 11/30
16/16 [==============================] - 2s 113ms/step - loss: 26.5799
Epoch 12/30
16/16 [==============================] - 2s 112ms/step - loss: 25.9843
Epoch 13/30


In [6]:
# train model

lr=1e-4

model_type='deep'
transfer=True
freeze_weights=False

margin=1.0

model_dir=os.path.join(dataset_dir,model_type)
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)

num_epochs=30

model=construct_model(model_type,input_dim,loss_type,num_classes,transfer,freeze_weights)
loss=construct_loss(loss_type,margin)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss=loss)

# Prepare callbacks for model saving and for learning rate adjustment.
filepath=os.path.join(model_dir,'final_%s_%s_transfer_%s_freeze_%s_margin_%s.h5'%(loss_type,model_type,transfer,freeze_weights,margin))
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

callbacks = [checkpoint]

history = model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=test_dataset,
    callbacks=callbacks)

plt.figure()
plt.plot(np.arange(num_epochs),history.history['loss'],'r',label='train_loss')
plt.plot(np.arange(num_epochs),history.history['val_loss'],'b',label='val_loss')
plt.legend()
filepath=os.path.join(model_dir,'final_loss_plot_%s_%s_transfer_%s_freeze_%s_margin_%s.png'%(loss_type,model_type,transfer,freeze_weights,margin))
plt.savefig(filepath)
plt.close()

Epoch 1/30
   1563/Unknown - 75s 48ms/step - loss: 1.7059
Epoch 00001: val_loss improved from inf to 1.00551, saving model to drive/My Drive/Colab Notebooks/cifar10/deep/final_classification_deep_transfer_True_freeze_False_margin_1.0.h5
1563/1563 [==============================] - 81s 52ms/step - loss: 1.7059 - val_loss: 0.0000e+00
Epoch 2/30
311/313 [============================>.] - ETA: 0s - loss: 1.1943
Epoch 00002: val_loss improved from 1.00551 to 0.79665, saving model to drive/My Drive/Colab Notebooks/cifar10/deep/final_classification_deep_transfer_True_freeze_False_margin_1.0.h5
1563/1563 [==============================] - 71s 46ms/step - loss: 1.0900 - val_loss: 0.7967
Epoch 3/30
312/313 [============================>.] - ETA: 0s - loss: 0.8983
Epoch 00003: val_loss improved from 0.79665 to 0.69601, saving model to drive/My Drive/Colab Notebooks/cifar10/deep/final_classification_deep_transfer_True_freeze_False_margin_1.0.h5
1563/1563 [==============================] - 72s 46ms

In [39]:
# tune margin

lr=1e-3
margins=[0.2,1]

model_type='deep'
classification=False
transfer=True
freeze_weights=False

loss_type='triplet'
l2_normalize=1 if loss_type=='triplet' else 0

model_dir=os.path.join(dataset_dir,model_type)
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)

num_epochs=25

fig,ax=plt.subplots(1,len(margins))
ax=ax.ravel()
for margin_index,margin in enumerate(margins):

  model=construct_model(model_type,input_dim,classification,num_classes,transfer,freeze_weights,l2_normalize)
  loss=construct_loss(loss_type,margin)

  # Compile the model
  model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss=loss)

  history = model.fit(
      train_dataset,
      epochs=num_epochs)

  ax[margin_index].set_title('Loss for margin_%s'%(str(margin)))
  ax[margin_index].plot(np.arange(num_epochs),history.history['loss'],'r',label='train_loss_margin_%s'%(str(margin)))
  
plt.legend()
plt.savefig(os.path.join(model_dir,'loss_plot_margin_%s_transfer_%s_freeze_%s_%s_%s.png'%(model_type,transfer,freeze_weights,loss_type,margin)))
plt.close()

Epoch 1/25
16/16 [==============================] - 10s 650ms/step - loss: 0.1618
Epoch 2/25
16/16 [==============================] - 2s 123ms/step - loss: 0.1519
Epoch 3/25
16/16 [==============================] - 2s 127ms/step - loss: 0.1404
Epoch 4/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1343
Epoch 5/25
16/16 [==============================] - 2s 125ms/step - loss: 0.1330
Epoch 6/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1303
Epoch 7/25
16/16 [==============================] - 2s 124ms/step - loss: 0.1263
Epoch 8/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1279
Epoch 9/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1290
Epoch 10/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1274
Epoch 11/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1227
Epoch 12/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1183
Epoch 13/25
16/16 [=====